In [2]:
import sys
import os
import numpy as np
import pandas as pd


In [9]:
# Détection du dossier d'install. de Distance
mcdsExe = 'MCDS.exe'
possVers = [7, 6]
possPaths = [os.path.join('C:\\', 'Program files (x86)'), os.path.join('C:', 'Program files')]
print('Recherche {} ...'.format(mcdsExe))
for path in possPaths:
    for ver in possVers:
        mcdsInstPath = os.path.join(path, 'Distance ' + str(ver))
        print(' - essai dans {} : '.format(mcdsInstPath), end='')
        if not os.path.exists(os.path.join(mcdsInstPath, mcdsExe)):
            mcdsInstPath = None
            print('non.')
        else:
            print('eureka !')
            break
    if mcdsInstPath:
        break

if mcdsInstPath:
    print('{} trouvé dans {}'.format(mcdsExe, mcdsInstPath))
else:
    print('Erreur : Impossible de trouver {}'.format(mcdsExe))

Recherche MCDS.exe ...
 - essai dans C:\Program files (x86)\Distance 7 : eureka !
MCDS.exe trouvé dans C:\Program files (x86)\Distance 7


In [ ]:
# Génération du fichier de commande
dataFileName = 'mcds-data.txt'


In [ ]:
# Génération du fichier de commande
outFileName = 'mcds-output.txt'
logFileName = 'mcds.log'
statsFileName = 'mcds-stats.txt'
bootFileName = 'mcds-bootstrap.txt'

cmdTxt = \
"""
{output}
{log}
{stats}
{bootstrap}
None
None
""".format(outFileName, logFileName, statsFileName, bootFileName,
           )

with open(cmdFileName, 'w') as cmdFile:
    cmdFile.write(cmdTxt)

In [ ]:
E:\perso\distance-sampling\PlateauFromages\JPM2017\CmdLine\output.tmp
E:\perso\distance-sampling\PlateauFromages\JPM2017\CmdLine\log.tmp
E:\perso\distance-sampling\PlateauFromages\JPM2017\CmdLine\stats.tmp
E:\perso\distance-sampling\PlateauFromages\JPM2017\CmdLine\bootstrap.tmp
None
None
Options;
Type=Point;
Distance=Radial /Measure='Meter';
Area /Units='Hectare';
Object=Single;
SF=1;
Selection=Sequential;
Lookahead=1;
Maxterms=5;
Confidence=95;
Print=Selection;
End;
Data /Structure=Flat;
Fields=STR_LABEL, STR_AREA, SMP_LABEL, SMP_EFFORT, DISTANCE;
Infile=E:\perso\distance-sampling\PlateauFromages\JPM2017\CmdLine\data.tmp /NoEcho;
End;
Estimate;
Distance;
Density=All;
Encounter=All;
Detection=All;
Size=All;
Estimator /Key=HA /Adjust=CO /Criterion=AIC;
Monotone=Strict;
Pick=AIC;
GOF;
Cluster /Bias=GXLOG;
VarN=Empirical;
End;
